In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

In [2]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_ML\FeaturesLocal\FeaturesST"

## <span style="color:yellow"> **Preparamos los Sets de Training** </span> 

In [3]:
loc_ts = r"D:\FacundoTorraca\Documents\TP2_ML\Training Sets"

In [4]:
evt_trn = pd.read_csv( loc_ts + "\\evt_18_20.csv" ); 
evt_tst = pd.read_csv( loc_ts + "\\evt_21_23.csv" );  

## <span style="color:yellow"> **Preparamos el ref_hash de cada ventana** </span> 

In [5]:
loc_lb = r"D:\FacundoTorraca\Documents\TP2_ML\Labels"

In [6]:
lb_ins_trn = pd.read_csv( loc_lb + "\\label_auc_21_23.csv" );
lb_ins_tst = pd.read_csv( loc_lb + "\\label_auc_24_26.csv" ); 

In [7]:
rh_trn = lb_ins_trn[ ["ref_hash"] ]
rh_tst = lb_ins_tst[ ["ref_hash"] ]

tg_trn = lb_auc_trn[ ["24_26_st"] ]
tg_tst = lb_auc_tst[ ["24_26_st"] ]

## <span style="color:yellow"> **=======================================================================================================** </span> 

## <span style="color:green"> **Cantidad de eventos por dispositivo en la ventana previa a la conversion** </span> 

In [9]:
cant_evt_trn = rh_trn.copy()
cant_evt_tst = rh_tst.copy()

cant_evt_trn = cant_evt_trn.merge( evt_trn["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_evt", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_evt_tst = cant_evt_tst.merge( evt_tst["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_evt", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_evt_trn.fillna( 0, inplace = True )
cant_evt_tst.fillna( 0, inplace = True )

cant_evt_trn.to_csv( loc_ftr + "\\cant_evt_trn.csv", index = False )
cant_evt_tst.to_csv( loc_ftr + "\\cant_evt_tst.csv", index = False )

## <span style="color:green"> **Cantidad de eventos atribuidos por dispositivo** </span> 

In [10]:
cevt_atr_trn = rh_trn.copy()
cevt_atr_tst = rh_tst.copy()

cant_atr_evt_trn = evt_trn[["ref_hash", "attributed"]]; cant_atr_evt_trn["attributed"] = cant_atr_evt_trn["attributed"].apply( lambda x: 1 if x else 0 );
cant_atr_evt_tst = evt_tst[["ref_hash", "attributed"]]; cant_atr_evt_tst["attributed"] = cant_atr_evt_tst["attributed"].apply( lambda x: 1 if x else 0 );

cant_atr_evt_trn = cant_atr_evt_trn.groupby( "ref_hash" ).agg( "sum" )
cant_atr_evt_tst = cant_atr_evt_tst.groupby( "ref_hash" ).agg( "sum" )

cevt_atr_trn = cevt_atr_trn.merge( cant_atr_evt_trn, how = "left", on = "ref_hash" ).fillna( 0 )
cevt_atr_tst = cevt_atr_tst.merge( cant_atr_evt_tst, how = "left", on = "ref_hash" ).fillna( 0 )

cevt_atr_trn.to_csv( loc_ftr + "\\cevt_atr_trn.csv", index = False )
cevt_atr_tst.to_csv( loc_ftr + "\\cevt_atr_tst.csv", index = False )

## <span style="color:green"> **Cantidad de eventos realizados con WiFi** </span> 

In [11]:
wifi_evt_trn = rh_trn.copy()
wifi_evt_tst = rh_tst.copy()

cant_wifi_evt_trn = evt_trn[["ref_hash", "wifi"]]; cant_wifi_evt_trn["wifi"] = cant_wifi_evt_trn["wifi"].apply( lambda x: 1 if x else 0 );
cant_wifi_evt_tst = evt_tst[["ref_hash", "wifi"]]; cant_wifi_evt_tst["wifi"] = cant_wifi_evt_tst["wifi"].apply( lambda x: 1 if x else 0 );

cant_wifi_evt_trn = cant_wifi_evt_trn.groupby( "ref_hash" ).agg( "sum" )
cant_wifi_evt_tst = cant_wifi_evt_tst.groupby( "ref_hash" ).agg( "sum" )

wifi_evt_trn = wifi_evt_trn.merge( cant_wifi_evt_trn, how = "left", on = "ref_hash" ).fillna( 0 )
wifi_evt_tst = wifi_evt_tst.merge( cant_wifi_evt_tst, how = "left", on = "ref_hash" ).fillna( 0 )

wifi_evt_trn.to_csv( loc_ftr + "\\wifi_evt_trn.csv", index = False )
wifi_evt_tst.to_csv( loc_ftr + "\\wifi_evt_tst.csv", index = False )

## <span style="color:green"> **Horario del primer evento en ese ventana** </span> 

Al agregarle la transformacion senoidal para agregarle perdiodicidad a la hora, la prediccion del algoritmo fue peor por lo que decidimos no agregarla

In [18]:
hr_f_evt_trn = rh_trn.copy()
hr_f_evt_tst = rh_tst.copy()

first_evt_hour_trn = evt_trn[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_evt_hour_tst = evt_tst[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

first_evt_hour_trn["time_to_frt_evt"] = ( pd.to_datetime( first_evt_hour_trn["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()
first_evt_hour_tst["time_to_frt_evt"] = ( pd.to_datetime( first_evt_hour_tst["date"] ) -  dt.datetime( year = 2019, month = 4, day = 24 ) ).dt.total_seconds()

first_evt_hour_trn["hour_frt_evt"] = pd.to_datetime( first_evt_hour_trn["date"] ).dt.hour
first_evt_hour_tst["hour_frt_evt"] = pd.to_datetime( first_evt_hour_tst["date"] ).dt.hour

first_evt_hour_trn.drop( ["date", "time_to_frt_evt"], axis = 1, inplace = True )
first_evt_hour_tst.drop( ["date", "time_to_frt_evt"], axis = 1, inplace = True )

hr_f_evt_trn = hr_f_evt_trn.merge( first_evt_hour_trn, how = "left", on = "ref_hash" )
hr_f_evt_tst = hr_f_evt_tst.merge( first_evt_hour_tst, how = "left", on = "ref_hash" )

#hr_f_evt_trn["hour_frt_evt"] = hr_f_evt_trn["hour_frt_evt"].apply( lambda x: np.sin( (x *np.pi)/24 ) )
#hr_f_evt_tst["hour_frt_evt"] = hr_f_evt_tst["hour_frt_evt"].apply( lambda x: np.sin( (x *np.pi)/24 ) )

hr_f_evt_trn.to_csv( loc_ftr + "\\hr_f_evt_trn.csv", index = False )
hr_f_evt_tst.to_csv( loc_ftr + "\\hr_f_evt_tst.csv", index = False )

## <span style="color:green"> **Hora mas popular del usuario en los eventos** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. 

In [13]:
main_ehr_trn = rh_trn.copy()
main_ehr_tst = rh_tst.copy()

hour_mode_trn = evt_trn[["ref_hash"]]; hour_mode_trn["hour"] = pd.to_datetime( evt_trn["date"] ).dt.hour
hour_mode_tst = evt_tst[["ref_hash"]]; hour_mode_tst["hour"] = pd.to_datetime( evt_tst["date"] ).dt.hour

hour_mode_trn = hour_mode_trn.groupby( ["ref_hash", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()
hour_mode_tst = hour_mode_tst.groupby( ["ref_hash", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()

hour_mode_trn = hour_mode_trn.sort_values( ["ref_hash","count"], ascending = False ).drop_duplicates( subset = ["ref_hash"], keep = "first" ).drop("count",axis = 1)
hour_mode_tst = hour_mode_tst.sort_values( ["ref_hash","count"], ascending = False ).drop_duplicates( subset = ["ref_hash"], keep = "first" ).drop("count",axis = 1)

main_ehr_trn = main_ehr_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )
main_ehr_tst = main_ehr_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )

main_ehr_trn["main_hour"] =  main_ehr_trn["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )
main_ehr_tst["main_hour"] =  main_ehr_tst["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )

main_ehr_trn.to_csv( loc_ftr + "\\main_ehr_trn.csv", index = False )
main_ehr_tst.to_csv( loc_ftr + "\\main_ehr_tst.csv", index = False )

## <span style="color:green"> **Tiempo hasta el primer evento** </span> 

In [17]:
frst_evt_trn = rh_trn.copy()
frst_evt_tst = rh_tst.copy()

first_event_rh_auc_trn = evt_trn[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_event_rh_auc_tst = evt_tst[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

first_event_rh_auc_trn["time_to_frt_evt"] = ( pd.to_datetime( first_event_rh_auc_trn["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()
first_event_rh_auc_tst["time_to_frt_evt"] = ( pd.to_datetime( first_event_rh_auc_tst["date"] ) -  dt.datetime( year = 2019, month = 4, day = 24 ) ).dt.total_seconds()

first_event_rh_auc_trn.drop( "date", axis = 1, inplace = True )
first_event_rh_auc_tst.drop( "date", axis = 1, inplace = True )

frst_evt_trn = frst_evt_trn.merge( first_event_rh_auc_trn, how = "left", on = "ref_hash" )
frst_evt_tst = frst_evt_tst.merge( first_event_rh_auc_tst, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_evt_trn.fillna( 3 * 24 * 3600, inplace = True )
frst_evt_tst.fillna( 3 * 24 * 3600, inplace = True )

frst_evt_trn.to_csv( loc_ftr + "\\frst_evt_trn.csv", index = False )
frst_evt_tst.to_csv( loc_ftr + "\\frst_evt_tst.csv", index = False )

## <span style="color:green"> **ID de evento mas realizado por el dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Lo codificamos haciendo la cantidad de eventos que se realizaron de ese id sobre el total de eventos realizados)

In [19]:
event_id_trn = rh_trn.copy()
event_id_tst = rh_tst.copy()

main_evt_id_trn = evt_trn.groupby( by = ["ref_hash","event_id"] ).agg( {"event_id":"count"} ).rename( columns = {"event_id":"cant_evt"} ).reset_index()
main_evt_id_tst = evt_tst.groupby( by = ["ref_hash","event_id"] ).agg( {"event_id":"count"} ).rename( columns = {"event_id":"cant_evt"} ).reset_index()

main_evt_id_trn = main_evt_id_trn.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( main_evt_id_trn["cant_evt"] )
main_evt_id_tst = main_evt_id_tst.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( main_evt_id_tst["cant_evt"] ) 

event_id_trn = event_id_trn.merge( main_evt_id_trn, how = "left", on = "ref_hash" )
event_id_tst = event_id_tst.merge( main_evt_id_tst, how = "left", on = "ref_hash" )

cant_event_id_trn = evt_trn[["ref_hash","event_id"]].groupby( "event_id" ).agg("count").reset_index().rename( columns = {"ref_hash":"cant_id"} )
cant_event_id_tst = evt_tst[["ref_hash","event_id"]].groupby( "event_id" ).agg("count").reset_index().rename( columns = {"ref_hash":"cant_id"} )

event_id_trn = event_id_trn.merge( cant_event_id_trn, how = "left", on = "event_id" ).drop( "event_id", axis = 1 )
event_id_tst = event_id_tst.merge( cant_event_id_tst, how = "left", on = "event_id" ).drop( "event_id", axis = 1 )

event_id_trn["evt_id_mean"] = event_id_trn["cant_id"] / len(evt_trn); event_id_trn.fillna( event_id_trn["evt_id_mean"].isnull().sum() / len(evt_trn), inplace = True ); del(event_id_trn["evt_id_mean"])
event_id_tst["evt_id_mean"] = event_id_tst["cant_id"] / len(evt_tst); event_id_tst.fillna( event_id_tst["evt_id_mean"].isnull().sum() / len(evt_tst), inplace = True ); del(event_id_tst["evt_id_mean"])

event_id_trn.to_csv( loc_ftr + "\\event_id_trn.csv", index = False )
event_id_tst.to_csv( loc_ftr + "\\event_id_tst.csv", index = False )

## <span style="color:green"> **Aplicacion que mas recibio eventos por cada dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Codificiamos sobre cuantas veces la aplicacion fue la principal de un dispositivo sobre el total de dispositivos)

In [20]:
mapp_evt_trn = rh_trn.copy()
mapp_evt_tst = rh_tst.copy()

app_mas_evt_trn = evt_trn.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_evt"} ).reset_index()
app_mas_evt_tst = evt_tst.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_evt"} ).reset_index()

app_mas_evt_trn = app_mas_evt_trn.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_evt_trn["cant_evt"] )
app_mas_evt_tst = app_mas_evt_tst.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_evt_tst["cant_evt"] ) 

mapp_evt_trn = mapp_evt_trn.merge( app_mas_evt_trn, how = "left", on = "ref_hash" )
mapp_evt_tst = mapp_evt_tst.merge( app_mas_evt_tst, how = "left", on = "ref_hash" )

mapp_evt_trn["to_count"] = 1; mapp_evt_trn["application_id"] = mapp_evt_trn[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(mapp_evt_trn); del(mapp_evt_trn["to_count"])
mapp_evt_tst["to_count"] = 1; mapp_evt_tst["application_id"] = mapp_evt_tst[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(mapp_evt_tst); del(mapp_evt_tst["to_count"])

#Completamos los NaNs con el promedio de su cantidad
mapp_evt_trn["application_id"].fillna( mapp_evt_trn["application_id"].isnull().sum() / len( mapp_evt_trn ), inplace = True )
mapp_evt_tst["application_id"].fillna( mapp_evt_tst["application_id"].isnull().sum() / len( mapp_evt_tst ), inplace = True )

mapp_evt_trn.to_csv( loc_ftr + "\\mapp_evt_trn.csv", index = False )
mapp_evt_tst.to_csv( loc_ftr + "\\mapp_evt_tst.csv", index = False )

## <span style="color:green"> **Tiempo hasta la ultimo evento en esa ventana** </span> 

In [25]:
last_evt_trn = rh_trn.copy()
last_evt_tst = rh_tst.copy()

last_event_rh_auc_trn = evt_trn[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )
last_event_rh_auc_tst = evt_tst[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )

last_event_rh_auc_trn["time_to_lst_evt"] = ( pd.to_datetime( last_event_rh_auc_trn["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()
last_event_rh_auc_tst["time_to_lst_evt"] = ( pd.to_datetime( last_event_rh_auc_tst["date"] ) -  dt.datetime( year = 2019, month = 4, day = 24 ) ).dt.total_seconds()

last_event_rh_auc_trn.drop( "date", axis = 1, inplace = True )
last_event_rh_auc_tst.drop( "date", axis = 1, inplace = True )

last_evt_trn = last_evt_trn.merge( last_event_rh_auc_trn, how = "left", on = "ref_hash" )
last_evt_tst = last_evt_tst.merge( last_event_rh_auc_tst, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
last_evt_trn.fillna( 3 * 24 * 3600, inplace = True )
last_evt_tst.fillna( 3 * 24 * 3600, inplace = True )

last_evt_trn.to_csv( loc_ftr + "\\last_evt_trn.csv", index = False )
last_evt_tst.to_csv( loc_ftr + "\\last_evt_tst.csv", index = False )

## <span style="color:green"> **Hizo eventos entre 21 hs y 3 hs (Noche)** </span>

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [26]:
evt_trn['date'] = pd.to_datetime(evt_trn['date'])
evt_tst['date'] = pd.to_datetime(evt_tst['date'])

evt_trn['evt_21_3'] = (evt_trn['date'].dt.hour < 4) | (evt_trn['date'].dt.hour > 20)
evt_tst['evt_21_3'] = (evt_tst['date'].dt.hour < 4) | (evt_tst['date'].dt.hour > 20)

evt_night_trn = rh_trn.copy()
evt_night_tst = rh_tst.copy()

hour_mode_trn = evt_trn.groupby('ref_hash').agg({'evt_21_3':'sum'}).reset_index()
hour_mode_tst = evt_tst.groupby('ref_hash').agg({'evt_21_3':'sum'}).reset_index()

evt_night_trn = evt_night_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
evt_night_tst = evt_night_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

evt_night_trn["evt_21_3"] =  (evt_night_trn["evt_21_3"] > 1).astype('int8')
evt_night_tst["evt_21_3"] =  (evt_night_tst["evt_21_3"] > 1).astype('int8')

evt_night_trn.to_csv( loc_ftr + "\\evt_nght_trn.csv", index = False )
evt_night_tst.to_csv( loc_ftr + "\\evt_nght_tst.csv", index = False )

del evt_trn['evt_21_3']
del evt_tst['evt_21_3']

## <span style="color:green"> **Hizo eventos entre 4 hs y 10 hs (Mañana)** </span>

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [27]:
evt_trn['date'] = pd.to_datetime(evt_trn['date'])
evt_tst['date'] = pd.to_datetime(evt_tst['date'])

evt_trn['evt_4_10'] = (evt_trn['date'].dt.hour < 11) & (evt_trn['date'].dt.hour > 3)
evt_tst['evt_4_10'] = (evt_tst['date'].dt.hour < 11) & (evt_tst['date'].dt.hour > 3)

evt_morn_trn = rh_trn.copy()
evt_morn_tst = rh_tst.copy()

hour_mode_trn = evt_trn.groupby('ref_hash').agg({'evt_4_10':'sum'}).reset_index()
hour_mode_tst = evt_tst.groupby('ref_hash').agg({'evt_4_10':'sum'}).reset_index()

evt_morn_trn = evt_morn_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
evt_morn_tst = evt_morn_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

evt_morn_trn["evt_4_10"] =  (evt_morn_trn["evt_4_10"] > 1).astype('int8')
evt_morn_tst["evt_4_10"] =  (evt_morn_tst["evt_4_10"] > 1).astype('int8')

evt_morn_trn.to_csv( loc_ftr + "\\evt_morn_trn.csv", index = False )
evt_morn_tst.to_csv( loc_ftr + "\\evt_morn_tst.csv", index = False )

del evt_trn['evt_4_10']
del evt_tst['evt_4_10']

## <span style="color:green"> **Hizo eventos entre 11 hs y 15 hs (Mediodia)** </span>

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [28]:
evt_trn['date'] = pd.to_datetime(evt_trn['date'])
evt_tst['date'] = pd.to_datetime(evt_tst['date'])

evt_trn['evt_11_15'] = (evt_trn['date'].dt.hour < 16) & (evt_trn['date'].dt.hour > 10)
evt_tst['evt_11_15'] = (evt_tst['date'].dt.hour < 16) & (evt_tst['date'].dt.hour > 10)

evt_midday_trn = rh_trn.copy()
evt_midday_tst = rh_tst.copy()

hour_mode_trn = evt_trn.groupby('ref_hash').agg({'evt_11_15':'sum'}).reset_index()
hour_mode_tst = evt_tst.groupby('ref_hash').agg({'evt_11_15':'sum'}).reset_index()

evt_midday_trn = evt_midday_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
evt_midday_tst = evt_midday_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

evt_midday_trn["evt_11_15"] =  (evt_midday_trn["evt_11_15"] > 1).astype('int8')
evt_midday_tst["evt_11_15"] =  (evt_midday_tst["evt_11_15"] > 1).astype('int8')

evt_midday_trn.to_csv( loc_ftr + "\\evt_mday_trn.csv", index = False )
evt_midday_tst.to_csv( loc_ftr + "\\evt_mday_tst.csv", index = False )

del evt_trn['evt_11_15']
del evt_tst['evt_11_15']

## <span style="color:green"> **Hizo eventos entre 16 hs y 20 hs (Tarde)** </span>

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [30]:
evt_trn['date'] = pd.to_datetime(evt_trn['date'])
evt_tst['date'] = pd.to_datetime(evt_tst['date'])

evt_trn['evt_16_20'] = (evt_trn['date'].dt.hour < 21) & (evt_trn['date'].dt.hour > 15)
evt_tst['evt_16_20'] = (evt_tst['date'].dt.hour < 21) & (evt_tst['date'].dt.hour > 15)

evt_after_trn = rh_trn.copy()
evt_after_tst = rh_tst.copy()

hour_mode_trn = evt_trn.groupby('ref_hash').agg({'evt_16_20':'sum'}).reset_index()
hour_mode_tst = evt_tst.groupby('ref_hash').agg({'evt_16_20':'sum'}).reset_index()

evt_after_trn = evt_after_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
evt_after_tst = evt_after_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

evt_after_trn["evt_16_20"] =  (evt_after_trn["evt_16_20"] > 1).astype('int8')
evt_after_tst["evt_16_20"] =  (evt_after_tst["evt_16_20"] > 1).astype('int8')

evt_after_trn.to_csv( loc_ftr + "\\evt_aftr_trn.csv", index = False )
evt_after_tst.to_csv( loc_ftr + "\\evt_aftr_tst.csv", index = False )

del evt_trn['evt_16_20']
del evt_tst['evt_16_20']

## <span style="color:green"> **Tipo de evento mas realizado por el dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Lo codificamos haciendo la cantidad de eventos que se realizaron de ese tipo sobre el total de eventos realizados)

In [31]:
kind_evt_trn = rh_trn.copy()
kind_evt_tst = rh_tst.copy()

kind_evt_trn = rh_trn.copy()
kind_evt_tst = rh_tst.copy()

main_kind_evt_trn = evt_trn.groupby( by = ["ref_hash","kind"] ).agg( {"kind":"count"} ).rename( columns = {"kind":"cant_evt"} ).reset_index()
main_kind_evt_tst = evt_tst.groupby( by = ["ref_hash","kind"] ).agg( {"kind":"count"} ).rename( columns = {"kind":"cant_evt"} ).reset_index()

main_kind_evt_trn = main_kind_evt_trn.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( main_kind_evt_trn["cant_evt"] )
main_kind_evt_tst = main_kind_evt_tst.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( main_kind_evt_tst["cant_evt"] ) 

kind_evt_trn = kind_evt_trn.merge( main_kind_evt_trn, how = "left", on = "ref_hash" )
kind_evt_tst = kind_evt_tst.merge( main_kind_evt_tst, how = "left", on = "ref_hash" )

cant_kind_evt_trn = evt_trn[["ref_hash","kind"]].groupby( "kind" ).agg("count").reset_index().rename( columns = {"ref_hash":"cant_kind"} )
cant_kind_evt_tst = evt_tst[["ref_hash","kind"]].groupby( "kind" ).agg("count").reset_index().rename( columns = {"ref_hash":"cant_kind"} )

kind_evt_trn = kind_evt_trn.merge( cant_kind_evt_trn, how = "left", on = "kind" ).drop( "kind", axis = 1 )
kind_evt_tst = kind_evt_tst.merge( cant_kind_evt_tst, how = "left", on = "kind" ).drop( "kind", axis = 1 )

kind_evt_trn["kind_mean"] = kind_evt_trn["cant_kind"] / len(evt_trn); kind_evt_trn.fillna( kind_evt_trn["kind_mean"].isnull().sum() / len(evt_trn), inplace = True ); del(kind_evt_trn["cant_kind"])
kind_evt_tst["kind_mean"] = kind_evt_tst["cant_kind"] / len(evt_tst); kind_evt_tst.fillna( kind_evt_tst["kind_mean"].isnull().sum() / len(evt_tst), inplace = True ); del(kind_evt_tst["cant_kind"])

kind_evt_trn.to_csv( loc_ftr + "\\kind_evt_trn.csv", index = False )
kind_evt_tst.to_csv( loc_ftr + "\\kind_evt_tst.csv", index = False )